# COGS 108 - Final Project (change this to your project's title)

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that student names will be included (but PIDs will be scraped from any groups who include their PIDs).

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Nilou Shahbandi
- Leonardo Gonzalez
- Brian Chen
- Tin-Trung Pham
- Zhongkang(ken) Fang

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

<a id='background'></a>

## Background & Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

### *Dataset 1*

(Copy this information for each dataset)
- Dataset Name: annual_aqi_by_county_2010
- Link to the dataset: https://aqs.epa.gov/aqsweb/airdata/download_files.html
- Number of observations: 999


- data contains AQI, calculated each day for each monitor for the Criteria Gases and PM10 and PM2.5 (FRM and non FRM).
- of Days CO, NO2, Ozone, SO2, PM2.5, PM10 where the daily monitor criteria were met.

- We extract the year of 2010 from the cancer data and merge with this data.
- We will also add more years differentiate by colums and merge with cancer data which already have different years.


### *Dataset 2*

(Copy this information for each dataset)
- Dataset Name: Lung and Bronchus Cancer Incidence in California
- Link to the dataset:[California Cancer Registry](https://explorer.ccrcal.org/application.html?site=47&data_type=1&graph_type=13&compareBy=sex&chk_sex_1=1&race=1&age_range=9&hdn_stage=101&year=4&advopt_precision=4&advopt_display=2#tableWrap)
- Number of observations: 9306
- 2010 - 2018

This data shows us the counts of lung or bronchus cancer in individuals of all races and sexes by county in California. It also includes breakdowns by age with additional statistics regarding the data.

We plan to use this data in combination with the pollution data to track the impact of pollution on lung or bronchus cancer indcidents in individuals living in California. By comparing changes in pollution with changes in cancer incident counts.


### *Dataset 3*

(Copy this information for each dataset)
- Dataset Name: 
- Link to the dataset:
- Number of observations:

#### Description:
- 


#### If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.
- 

# Setup

In [1]:
## YOUR CODE HERE
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns

aqi_2010 = pd.read_csv('data/aqireport2010.csv')
county_cancer = pd.read_csv('data/countyCancer.csv')
asthma = pd.read_csv('data/Asthma.csv')

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
asthma.head()

,State,ChildCount,Percent,Year,AdultCount,Percent.1
0,Alabama,"141,500",13.20%,2014.0,"394,600",10.50%
1,Alaska,---,---,NaN,"50,700",9.20%
2,Arizona,"129,200",8.10%,2016.0,"545,100",10.00%
3,Arkansas,---,---,NaN,"225,900",9.80%
4,California,"542,700",6.20%,NaN,"2,607,600",8.50%


In [3]:
aqi_2010.head()

,County Code,County,# Days with AQI,Good,Moderate,Unhealthy for Sensitive Groups,Unhealthy,Very Unhealthy,Hazardous,AQI Maximum,AQI 90th Percentile,AQI Median,# Days CO,# Days NO2,# Days O3,# Days PM2.5,# Days PM10
0,6001,"Alameda County, CA",365,245,115,4,1,.,.,179,68.0,43.0,.,24,135,206,.
1,6005,"Amador County, CA",361,302,48,10,1,.,.,151,64.0,35.0,.,.,361,.,.
2,6007,"Butte County, CA",365,208,144,13,.,.,.,126,84.0,47.0,.,.,264,101,.
3,6009,"Calaveras County, CA",365,271,81,12,1,.,.,154,84.0,41.0,.,.,309,56,.
4,6011,"Colusa County, CA",353,322,30,1,.,.,.,119,49.0,38.0,.,.,305,43,5


In [8]:
county_cancer.head()

,Year,Sex,Race/Ethnicity,Age,California Region,Stage at Diagnosis,"Age-adjusted Rate per 100,000",Age-adjusted Rate Standard Error,Age-adjusted Rate Lower Confidence Interval,Age-adjusted Rate Upper Confidence Interval,Count
0,2010,Both Sexes,All Races,All Ages,Alameda County,All Stages,44.8118,1.8013,41.3504,48.4812,646
1,2010,Both Sexes,All Races,All Ages,Butte County,All Stages,63.0616,4.9021,53.8195,73.5358,171
2,2010,Both Sexes,All Races,All Ages,Contra Costa County,All Stages,50.2429,2.2114,46.0016,54.7691,536
3,2010,Both Sexes,All Races,All Ages,El Dorado County,All Stages,54.2364,4.9899,44.8983,65.1021,124
4,2010,Both Sexes,All Races,All Ages,Fresno County,All Stages,51.7782,2.6266,46.7576,57.1687,401


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
# Asthma

In [ ]:
# AQI


In [ ]:
# County cancer

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*